In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

# import json
# with open('./stage_5.json', 'r') as file:
#     data = json.load(file)

# print(data)
base_url = "https://digitalwires.dpa-newslab.com/iaeystP2ZVzUx1PBhd5Tr2LODbdSavCV/aufschaltung/au-clC0AH31tVaf0fbHu2/apigate-electionsdata/f-wmGZTrfmonUjD1FR"
us_election_id = "us-2024"
results = "results?election=us-2024&stage=live"

url = f"{base_url}/{results}"


response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Successfully received data")
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response content: {response.text}")

Successfully received data


In [2]:
results_per_constituency = data['election']['contest'][0]['results_per_constituency']
constituencies_data = data['constituencies']

# Create a dictionary to map constituency_id to name
constituency_id_to_name = {}
for constituency in constituencies_data:
    constituency_id = constituency['id']
    constituency_id_to_name[constituency_id] = constituency['name']

# Create lists to store the data for our DataFrame
ids = []
names = []

# Iterate through results_per_constituency
for result in results_per_constituency:
    constituency_id = result['constituency_id']
    ids.append(constituency_id)
    
    # Get the name from our mapping, or use 'Unknown' if not found
    name = constituency_id_to_name.get(constituency_id, 'Unknown')
    names.append(name)

# Create the DataFrame
constituencies_df = pd.DataFrame({
    'id': ids,
    'name': names
})

# Verify the length of the DataFrame
print(f"Hello of rows in df_constituencies: {len(constituencies_df)}")

Hello of rows in df_constituencies: 51


In [3]:
electoral_votes = {
    'alabama': 9, 'alaska': 3, 'arizona': 11, 'arkansas': 6, 'california': 54,
    'colorado': 10, 'connecticut': 7, 'delaware': 3, 'florida': 30, 'georgia': 16,
    'hawaii': 4, 'idaho': 4, 'illinois': 19, 'indiana': 11, 'iowa': 6,
    'kansas': 6, 'kentucky': 8, 'louisiana': 8, 'maine': 4, 'maryland': 10,
    'massachusetts': 11, 'michigan': 15, 'minnesota': 10, 'mississippi': 6,
    'missouri': 10, 'montana': 4, 'nebraska': 5, 'nevada': 6, 'new_hampshire': 4,
    'new_jersey': 14, 'new_mexico': 5, 'new_york': 28, 'north_carolina': 16,
    'north_dakota': 3, 'ohio': 17, 'oklahoma': 7, 'oregon': 8, 'pennsylvania': 19,
    'rhode_island': 4, 'south_carolina': 9, 'south_dakota': 3, 'tennessee': 11,
    'texas': 40, 'utah': 6, 'vermont': 3, 'virginia': 13, 'washington': 12,
    'washington_dc': 3, 'west_virginia': 4, 'wisconsin': 10, 'wyoming': 3
}

# First create the map data
results = data['election']['contest'][0]['results_per_constituency']
results_df = pd.DataFrame(columns=['state', 'position', 'seats_republican', 'seats_democrat', 'type'])

# Process each constituency
for result in results:
    constituency_id = result['constituency_id']
    state_name = constituencies_df.loc[constituencies_df['id'] == constituency_id, 'name'].iloc[0]
    
    # Initialize empty values
    position = 'battleground'  # Default position
    seats_republican = ''
    seats_democrat = ''
    result_type = ''
    
    # Only process seats if there are preliminary results
    if result['latest'] and result['latest']['type'] == 'preliminary':
        result_type = 'preliminary'
        
        for candidate_result in result['latest']['results']:
            target_id = candidate_result['target_id']
            
            # Find the corresponding candidate
            candidate = next((c for c in data['candidates'] if c['id'] == target_id), None)
            if candidate:
                person_id = candidate['person_id']
                person = next((p for p in data['persons'] if p['id'] == person_id), None)
                
                if person:
                    name = f"{person['first_name']} {person['last_name']}"
                    if 'Donald Trump' in name and candidate_result.get('seats'):
                        seats_republican = candidate_result['seats'].get('absolute', '')
                    elif 'Kamala Harris' in name and candidate_result.get('seats'):
                        seats_democrat = candidate_result['seats'].get('absolute', '')
        
        # Check for split states first
        if (seats_republican and seats_democrat and 
            float(seats_republican) > 0 and float(seats_democrat) > 0):
            position = 'split_called'
        # Then check for single winner states
        elif seats_republican and float(seats_republican) > 0:
            position = 'republican'
        elif seats_democrat and float(seats_democrat) > 0:
            position = 'democrat'
    
    new_row = pd.DataFrame({
        'state': [state_name],
        'position': [position],
        'seats_republican': [seats_republican],
        'seats_democrat': [seats_democrat],
        'type': [result_type]
    })
    
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Save map data
results_df.to_csv("dpa_map_data.csv", index=False)

# Initialize counters for widget
democrat_seats = 0
republican_seats = 0
battleground_seats = 0

# Process each state for widget data
for _, row in results_df.iterrows():
    state = row['state'].lower().replace(' ', '_')
    position = row['position'] if pd.notna(row['position']) else 'battleground'
    
    # Get electoral votes for this state
    state_electoral_votes = electoral_votes.get(state, 0)
    if state == 'district_of_columbia':  # Handle DC special case
        state_electoral_votes = electoral_votes['washington_dc']
    
    # Convert seats to numbers, handling empty strings and NaN
    seats_dem = int(float(row['seats_democrat'])) if pd.notna(row['seats_democrat']) and row['seats_democrat'] != '' else 0
    seats_rep = int(float(row['seats_republican'])) if pd.notna(row['seats_republican']) and row['seats_republican'] != '' else 0
    
    print(f"\nProcessing {state}")
    print(f"Position: {position}")
    print(f"Electoral votes: {state_electoral_votes}")
    print(f"Democrat seats: {seats_dem}, Republican seats: {seats_rep}")
    
    if position == 'democrat':
        if seats_dem > 0:
            democrat_seats += seats_dem
        else:
            democrat_seats += state_electoral_votes
            
    elif position == 'republican':
        if seats_rep > 0:
            republican_seats += seats_rep
        else:
            republican_seats += state_electoral_votes
            
    elif position == 'split_called':
        democrat_seats += seats_dem
        republican_seats += seats_rep
        # If there are remaining electoral votes, add to battleground
        remaining_votes = state_electoral_votes - (seats_dem + seats_rep)
        if remaining_votes > 0:
            battleground_seats += remaining_votes
            
    else:  # battleground
        battleground_seats += state_electoral_votes

print("\nFinal totals:")
print(f"Democrat: {democrat_seats}")
print(f"Republican: {republican_seats}")
print(f"Battleground: {battleground_seats}")
print(f"Total: {democrat_seats + republican_seats + battleground_seats}")

# Create widget DataFrame and save
widget_df = pd.DataFrame({
    'democrat': [int(democrat_seats)],
    'republican': [int(republican_seats)],
    'battleground': [int(battleground_seats)],
    'lean_harris': [0],  # Added to maintain compatible format
    'lean_trump': [0]    # Added to maintain compatible format
})

widget_df.to_csv("dpa_widget.csv", index=False)

In [9]:
# swing_states = ['michigan', 'wisconsin', 'pennsylvania', 'nevada', 'arizona', 'north_carolina', 'georgia']

# def update_state_data(state, results_row):
#     csv_name = f"dpa_{state.lower().replace(' ', '_')}.csv"
#     df = pd.read_csv(csv_name, parse_dates=['date'])
    
#     # Convert scores to float for proper comparison
#     trump_score = float(results_row['trump_score']) if results_row['trump_score'] != '' else 0
#     harris_score = float(results_row['harris_score']) if results_row['harris_score'] != '' else 0
    
#     new_row = pd.DataFrame({
#         'date': [datetime.now()],
#         'trump_score': [results_row['trump_score']],
#         'harris_score': [results_row['harris_score']],
#         'candidate_lead': ['harris' if harris_score > trump_score else 'trump' if trump_score > harris_score else ''],
#         'position': [results_row['position']]
#     })
    
#     df = pd.concat([df, new_row], ignore_index=True)
#     df.to_csv(csv_name, index=False)
#     print(f"Updated {csv_name}")

# def normalize_state_name(name):
#     return name.lower().replace(' ', '_')

# def update_swing_state_data(results_df):
#     results_df['normalized_state'] = results_df['state'].apply(normalize_state_name)
    
#     for state in swing_states:
#         csv_name = f"dpa_{state.lower().replace(' ', '_')}.csv"
        
#         # Check the latest position in the current CSV
#         current_df = pd.read_csv(csv_name)
#         if not current_df.empty and current_df['position'].iloc[-1] in ['democrat', 'republican']:
#             print(f"Skipping {state} as it's already decided ({current_df['position'].iloc[-1]})")
#             continue
        
#         # Find the corresponding row in results_df
#         state_data = results_df[results_df['normalized_state'] == normalize_state_name(state)]
        
#         if not state_data.empty:
#             update_state_data(state, state_data.iloc[0])
#         else:
#             print(f"No data found for {state} in results_df")

# update_swing_state_data(results_df)

Updated dpa_michigan.csv
Updated dpa_wisconsin.csv
Updated dpa_pennsylvania.csv
Updated dpa_nevada.csv
Updated dpa_arizona.csv
Updated dpa_north_carolina.csv
Updated dpa_georgia.csv


/var/folders/c8/bvk5bjs15mzg5h8m_jcbywx40000gn/T/ipykernel_83406/2288087970.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
/var/folders/c8/bvk5bjs15mzg5h8m_jcbywx40000gn/T/ipykernel_83406/2288087970.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
/var/folders/c8/bvk5bjs15mzg5h8m_jcbywx40000gn/T/ipykernel_83406/2288087970.py:19: FutureWarning: The behavior of DataFrame concatenation with e

In [8]:
# erase and re-create all csvs
# import os
# columns = ['date', 'trump_score', 'harris_score', 'candidate_lead', 'position']

# # Function to reset CSV files
# def reset_csv_files():
#     for state in swing_states:
#         filename = f"dpa_{state.lower().replace(' ', '_')}.csv"
#         # Remove existing file if it exists
#         if os.path.exists(filename):
#             os.remove(filename)
#         # Create new empty CSV file
#         df = pd.DataFrame(columns=columns)
#         df.to_csv(filename, index=False)
#         print(f"Reset and created empty CSV file: {filename}")

# reset_csv_files()

Reset and created empty CSV file: dpa_michigan.csv
Reset and created empty CSV file: dpa_wisconsin.csv
Reset and created empty CSV file: dpa_pennsylvania.csv
Reset and created empty CSV file: dpa_nevada.csv
Reset and created empty CSV file: dpa_arizona.csv
Reset and created empty CSV file: dpa_north_carolina.csv
Reset and created empty CSV file: dpa_georgia.csv
